In [20]:
import numpy as np
import pandas as pd

In [21]:
places = pd.read_csv('./kyoto-geocoded-3.csv')
places

,ID,Name,Address,lat,lng,Unnamed: 5,Unnamed: 6,open,close,open_time_min,close_time_min,Unnamed: 11,time_to_spend
0,9999999,京都駅,京都市下京区東塩小路釜殿町,34.980983,135.754836,530,2359,5:30:00,23:59:00,330,1439,0.0,0
1,1000002,慈照寺（銀閣寺）銀閣,京都市左京区銀閣寺町2,35.026864,135.798269,830,1700,8:30:00,17:00:00,510,1020,40.0,40
2,1000003,銀閣寺（慈照寺）東求堂,京都市左京区銀閣寺町2,35.026864,135.798269,830,1700,8:30:00,17:00:00,510,1020,40.0,40
3,1000010,下鴨神社（賀茂御祖神社）東本殿西本殿,京都市左京区下鴨泉川町59,35.034883,135.771905,630,1700,6:30:00,17:00:00,390,1020,30.0,30
4,1000017,東福寺三門,本町15丁目778,34.977097,135.774106,900,1600,9:00:00,16:00:00,540,960,20.0,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,7000011,名勝大沢池,右京区嵯峨大沢町４番地,35.028300,135.677945,600,1900,6:00:00,19:00:00,360,1140,45.0,45
80,7000064,知恩院紫雲水,東山区林下町400,35.006161,135.783806,900,1630,9:00:00,16:30:00,540,990,NaN,33
81,8000001,嵐山通船,南乗船場・京都市西京区嵐山山中町/北乗船場・京都市右京区嵯峨天竜寺芒ノ馬場町,35.009449,135.666773,900,1700,9:00:00,17:00:00,540,1020,30.0,30
82,8000022,知恩院七不思議,東山区林下町400,35.006161,135.783806,900,1630,9:00:00,16:30:00,540,990,NaN,33


In [22]:
time_to_spend = places.set_index('Name')['time_to_spend'].to_dict()
open_time_min = places.set_index('Name')['open_time_min'].to_dict()
close_time_min = places.set_index('Name')['close_time_min'].to_dict()
name_list = places['Name'].to_list()

In [23]:
def manhattan_dist(lat1, lng1, lat2, lng2):
    dist_x = np.abs(lng1 - lng2) * 91287.7885
    dist_y = np.abs(lat1 - lat2) * 110940.5844
    return dist_x + dist_y

In [24]:
matrix = dict()
for f_idx, f_row in places.iterrows():
    for t_idx, t_row in places.iterrows():
        distance = manhattan_dist(f_row.lat, f_row.lng, t_row.lat, t_row.lng)
        matrix[(f_row['Name'], t_row['Name'])] = (np.ceil(15 * distance / 1000))

In [25]:
class NodeManager:
    def __init__(self, names):
        self.names = names
        self.n = len(self.names)
        self.nodes = list(range(self.n))

        self.name_map_node = dict(zip(self.names, self.nodes))
        self.node_map_name = {v: k for k, v in self.name_map_node.items()}

    def node_to_name(self, node):
        return self.node_map_name[node]

    def name_to_node(self, name):
        return self.name_map_node[name]

In [26]:
node_manager = NodeManager(name_list)

In [27]:
from ortools.constraint_solver import pywrapcp

index_manager = pywrapcp.RoutingIndexManager(node_manager.n, 1, node_manager.name_to_node('京都駅'))

In [28]:
routing = pywrapcp.RoutingModel(index_manager)

In [29]:
def time_callback(from_index, to_index):
    from_node = index_manager.IndexToNode(from_index)
    to_node = index_manager.IndexToNode(to_index)
    from_name = node_manager.node_to_name(from_node)
    to_name = node_manager.node_to_name(to_node)

    return matrix[from_name, to_name] + time_to_spend[from_name]

In [30]:
transit_callback_index = routing.RegisterTransitCallback(time_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [31]:
for i in node_manager.names:
    if i == '京都駅':
        continue
    index = index_manager.NodeToIndex(node_manager.name_to_node(i))
    routing.AddDisjunction([index], 10_000)

In [32]:
routing.AddDimension(
    transit_callback_index,
    60,
    60 * 20,
    False,
    'Time'
)
time_dimension = routing.GetDimensionOrDie('Time')

In [33]:
for i in node_manager.names:
    index = index_manager.NodeToIndex(node_manager.name_to_node(i))
    open_time = 330.0 if np.isnan(open_time_min[i]) else open_time_min[i]
    close_time = 1439.0 if np.isnan(close_time_min[i]) else close_time_min[i]
    # print(open_time, close_time)
    time_dimension.CumulVar(index).SetRange(int(open_time), int(close_time))

In [34]:
from ortools.constraint_solver import routing_enums_pb2

search_params = pywrapcp.DefaultRoutingSearchParameters()
search_params.time_limit.seconds = 10
search_params.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)

In [35]:
solution = routing.SolveWithParameters(search_params)



In [38]:
import math

def minutes2hm(n):
    hour = math.floor(n / 60)
    min = n % 60
    return "{0}:{1:02d}".format(hour, min)

def print_solution(routing, solution):
    vehicle_id = 0
    index = routing.Start(vehicle_id)
    while not routing.IsEnd(index):
        previous_index = index
        name = node_manager.node_to_name(index_manager.IndexToNode(previous_index))
        output = name + ': '
        output += minutes2hm(solution.Value(time_dimension.CumulVar(index)))
        print(output)
        index = solution.Value(routing.NextVar(index))
    print('京都駅:', minutes2hm(solution.Value(time_dimension.CumulVar(index))))

print_solution(routing, solution)

京都駅: 5:30
水火天満宮菅公影向松: 7:07
下鴨神社（賀茂御祖神社）東本殿西本殿: 8:08
三福寺地蔵菩薩像: 9:22
天授庵絹本著色細川幽斎像同夫人像: 10:07
南禅寺方丈: 10:37
知恩院濡髪大明神: 11:28
知恩院七不思議: 11:38
知恩院紫雲水: 12:11
建仁寺方丈: 13:08
蓮光寺駒止地蔵菩薩像: 14:05
蓮光寺阿弥陀如来像: 14:15
南岩倉明王院不動寺空海作「石像不動明王」: 14:36
安養寺木造阿弥陀如来像: 14:53
廬山寺慈恵大師自筆遺告: 15:36
廬山寺木造如意輪観音半跏像: 15:46
廬山寺絹本著色普賢十羅刹女像: 15:56
北野天満宮影向松: 17:00
北野天満宮東門の椋: 17:30
京都駅: 19:51


In [63]:
lats = places.set_index('Name')['lat'].to_dict()
lngs = places.set_index('Name')['lng'].to_dict()

def solution_to_csv(routing, solution):
    vehicle_id = 0
    index = routing.Start(vehicle_id)
    result = []
    while not routing.IsEnd(index):
        previous_index = index
        name = node_manager.node_to_name(index_manager.IndexToNode(previous_index))
        time = minutes2hm(solution.Value(time_dimension.CumulVar(index)))
        lat = lats[name]
        lng = lngs[name]
        result.append([name, time, lat, lng])

        index = solution.Value(routing.NextVar(index))
    return pd.DataFrame(result, columns=['Name', 'Time', 'Lat', 'Lng'], index='Name')

res = solution_to_csv(routing, solution)

TypeError: Index(...) must be called with a collection of some kind, 'Name' was passed

In [64]:
res


,Name,Time,Lat,Lng
0,京都駅,5:30,34.980983,135.754836
1,水火天満宮菅公影向松,7:07,35.036738,135.752018
2,下鴨神社（賀茂御祖神社）東本殿西本殿,8:08,35.034883,135.771905
3,三福寺地蔵菩薩像,9:22,35.013157,135.777185
4,天授庵絹本著色細川幽斎像同夫人像,10:07,35.010271,135.791799
5,南禅寺方丈,10:37,35.010271,135.791799
6,知恩院濡髪大明神,11:28,35.006161,135.783806
7,知恩院七不思議,11:38,35.006161,135.783806
8,知恩院紫雲水,12:11,35.006161,135.783806
9,建仁寺方丈,13:08,35.000455,135.773639


In [65]:
res.to_csv('./kyoto-route.csv', index_label = ["ID"])

